# Lesson: Introduction to Creating and Updating Data in MongoDB

Welcome, stargazers! Today, we're venturing into the realms of creating and updating data in MongoDB. It bears a resemblance to a space diary, where you log observations of new celestial objects and note changes in their properties over time.
How to Insert Documents into a Collection

Our data logbooks in MongoDB are referred to as documents. Have you discovered a new star? Let's create a log entry, or document, for it:

In [ ]:
const mongoose = require('mongoose');
const starSchema = new mongoose.Schema({
  name: String, 
  constellation: String, 
  magnitude: Number
});
const Star = mongoose.model('Star', starSchema);

async function insertStar() {
  await mongoose.connect('mongodb://127.0.0.1/myDB');
  const star = new Star({
    name: "Polaris",
    constellation: "Ursa Minor",
    magnitude: 2.0
  });
  const result = await star.save();
  console.log(result);
  mongoose.connection.close();
}

insertStar();

Here, we created a star object instance from the Star model using the new Star call and saved it into the database with the save() async method. Simple, isn't it?

At the end of the function, we called mongoose.connection.close();. This line closes the current connection to the MongoDB server. This helps in freeing up system resources that our application was using during the connection. It's a good practice to close unnecessary connections especially when the DB operations are no longer required or when all the tasks are finished. This maintains optimal application performance and prevents memory and connection leaks.

For multiple discoveries, we can log them all at once using insertMany():

In [ ]:
async function insertStars() {
  await mongoose.connect('mongodb://127.0.0.1/myDB');
  const stars = [{
    name: "Sirius",
    constellation: "Canis Major",
    magnitude: -1.46
  },{
    name: "Canopus",
    constellation: "Carina",
    magnitude: -0.74
  },{
    name: "Arcturus",
    constellation: "Boötes",
    magnitude: -0.04
  }];
  const result = await Star.insertMany(stars);
  console.log(result);
  mongoose.connection.close();
}

insertStars();

After each insert operation, MongoDB provides a response, acknowledging the operation's successful completion.

### Updating Documents in a Collection

Data logging isn't limited to discovery; data needs to be updated based on new observations. For this, Mongoose provides the findOne() and save().

Assuming the magnitude of Polaris has been updated from 2.0 to 2.02, we can reflect this new observation in our MongoDB database:

In [ ]:
async function updateStar() {
  await mongoose.connect('mongodb://127.0.0.1/myDB');
  const star = await Star.findOne({ name: "Polaris" });
  if (!star) {
    console.log('Star not found.');
    return;
  }
  star.magnitude = 2.02;
  const result = await star.save();
  console.log(result);
  mongoose.connection.close();
}

updateStar();

// We fetch the document using findOne(), modify the magnitude, and save back to the database.

### Exercises: 

Stellar Navigator, our mission today involves both observing new celestial phenomena and refining our cosmic ledger. The starter code simulates a scenario where new stars, "Orionis" and "Polaris", are documented, and an existing record of "Polaris" is updated in our MongoDB using Mongoose. By running the React frontend app, we'll trigger these operations through API calls. Observe the frontend interacting with our server as we catalog the stars!

In [ ]:
// index.js

const mongoose = require('mongoose');
const express = require('express');
const cors = require('cors');
const app = express();
const initialDBSetup = require('./initialDBSetup');


app.use(cors());

const starSchema = new mongoose.Schema({
  name: String,
  constellation: String,
  magnitude: Number
});

const Star = mongoose.model('Star', starSchema);

// Connect to MongoDB
mongoose
  .connect('mongodb://127.0.0.1/myDB', { useNewUrlParser: true, useUnifiedTopology: true })
  .then(async () => {
    console.log('connected');
    // initialize DB
    await initialDBSetup();
  });
  
// Adding Orionis
app.get('/api/add-star-orionis', async (req, res) => {
  const orionis = new Star({ name: "Orionis", constellation: "Orion", magnitude: 3.5 });
  await orionis.save();
  res.send("Orionis has been added to the stars collection.");
});

// Adding Polaris
app.get('/api/add-star-polaris', async (req, res) => {
  const polaris = new Star({ name: "Polaris", constellation: "Ursa Minor", magnitude: 2.0 });
  await polaris.save();
  res.send("Polaris has been added to the stars collection.");
});

// Updating Polaris Magnitude
app.get('/api/update-polaris-magnitude', async (req, res) => {
  await Star.updateOne({ name: "Polaris" }, { $set: { magnitude: 2.02 } });
  res.send("Polaris magnitude updated to 2.02.");
});

// Start the server
const port = process.env.PORT || 5000;

app.listen(port, () => {
    console.log(`Server is running on port ${port}`);
});

In [ ]:
// App.js
import { useState } from 'react';
import axios from 'axios';

function App() {
  const [status, setStatus] = useState('');

  const addStars = () => {
    // Add Orionis
    axios.get('/api/add-star-orionis')
      .then(response => setStatus(prev => `${prev}\n${response.data}`));
    // Add Polaris
    axios.get('/api/add-star-polaris')
      .then(response => setStatus(prev => `${prev}\n${response.data}`));
  };
  
  const updatePolarisMagnitude = () => {
    axios.get('/api/update-polaris-magnitude')
      .then(response => setStatus(prev => `${prev}\n${response.data}`));
  };

  return (
    <div className="App">
      <h1>Astronomy Records</h1>
      <button onClick={addStars}>Add Stars</button>
      <button onClick={updatePolarisMagnitude}>Update Polaris Magnitude</button>
      <pre>Status: {status.trim()}</pre>
    </div>
  );
}

export default App;

Great job, Space Voyager! In this update, we're enhancing our understanding of MongoDB operations. You're tasked with adjusting the mass of Jupiter using Node.js and Mongoose. Based on the latest celestial data, Jupiter's mass has been revised. Use the server/index.js file for the backend operation.

In [ ]:
// index.js

const express = require('express');
const cors = require('cors');
const mongoose = require('mongoose');
const initialDBSetup = require('./initialDBSetup');

// Create a new express app
const app = express();

app.use(express.json());
app.use(cors({origin: 'http://localhost:3000'}));

const planetSchema = new mongoose.Schema({
  name: String,
  size: String,
  mass: Number,
  distanceFromSun: String
});

// Connect to MongoDB
mongoose
  .connect('mongodb://127.0.0.1/myDB', { useNewUrlParser: true, useUnifiedTopology: true })
  .then(async () => {
    console.log('connected');
    // initialize DB
    await initialDBSetup();
  });

const Planet = mongoose.model('Planet', planetSchema);

async function updatePlanetMass() {
  const planet = await Planet.findOne({ name: 'Jupiter' });
  if (planet) {
    // TODO: Change the 'mass' of the planet to 318
    planet.mass = 318
    // TODO: Don't forget to save the updated planet
    planet.save()
    return `Updated Jupiter mass to ${planet.mass}`;
  } else {
    return 'Jupiter not found.';
  }
}

app.get('/api/some-endpoint', async (req, res) => {
  const message = await updatePlanetMass();
  res.json({message});
});

// Start the server
const port = process.env.PORT || 5000;

app.listen(port, () => {
    console.log(`Server is running on port ${port}`);
});


Space Voyager, during your last trip, you observed a new planet and attempted to log its details in the universe catalog. However, the new entry isn't getting added. Your mission is to ensure this new celestial body is properly recorded by fixing the code!

In [ ]:
const express = require('express');
const cors = require('cors');
const mongoose = require('mongoose');
const initialDBSetup = require('./initialDBSetup');

// Create a new express app
const app = express();

app.use(express.json());
app.use(cors({origin: 'http://localhost:3000'}));

// Connect to MongoDB
mongoose
  .connect('mongodb://127.0.0.1/myDB', { useNewUrlParser: true, useUnifiedTopology: true })
  .then(async () => {
    console.log('connected');
    // initialize DB
    await initialDBSetup();
  });

const planetSchema = new mongoose.Schema({
  name: String,
  size: String,
  mass: Number,
  distanceFromSun: String
});

const Planet = mongoose.model('Planet', planetSchema);

app.get('/api/some-endpoint', async (req, res) => {
  const newPlanet = new Planet({
    name: 'Pluto',
    size: 'Dwarf',
    mass: 0.0022,
    distanceFromSun: '5.9 billion km'
  });

  try {
    await newPlanet.save(); // changed from savePlanet() to save()
    const result = await Planet.find({ name: 'Pluto'});
    res.status(200).json({ message: 'Planet added: ' + result.toString(), data: result });
  } catch (error) {
    res.status(500).json({ message: 'Error adding planet' });
  }
});

app.listen(5000, () => {
  console.log('Server running on port 5000');
});

Great job, Space Voyager! Now it's your turn to add some of the stars yourself. Write the lines of code that will connect to create a new planet based on the client input

In [ ]:
// index.js - express server to create a new planet
const express = require('express');
const mongoose = require('mongoose');
const bodyParser = require('body-parser');
const app = express();
const port = 5000;
mongoose.connect('mongodb://127.0.0.1/myDB');

const planetSchema = new mongoose.Schema({
  name: String,
  size: String,
  mass: Number,
  distanceFromSun: String
}, { versionKey: false });

const Planet = mongoose.model('Planet', planetSchema);

app.use(bodyParser.json());
app.use(bodyParser.urlencoded({ extended: true }));

app.post('/api/planets', async (req, res) => {
  if (!req.body.name) {
    return res.status(400).send('Name is required');
  }
  // TODO: Create a new planet with the given name req.body.name using Mongoose Model
  const newPlanet = new Planet({
    name: req.body.name
  })
  
  try {
    await newPlanet.save(); 
    const result = await Planet.find({ name: 'Pluto'}); // Save the new planet and return 200 status code with the newly created object
    res.status(200).json({ message: 'Planet added: ' + result.toString(), data: result });
  } catch (error) {
    res.status(500).json({ message: 'Error adding planet' }); // Return 500 status code with relevant error message in case of failure
  }
  
});

app.listen(port, () => {
  console.log(`Server running at http://localhost:${port}`);
});

In [ ]:
// App.js - React frontend to create a new planet
import { useState } from 'react';
import axios from 'axios';

function App() {
  const [planet, setPlanet] = useState('');
  const [message, setMessage] = useState('');

  const addPlanet = async () => {
    try {
      const response = await axios.post('/api/planets', { name: planet });
      setMessage('Planet added successfully: ' + JSON.stringify(response.data));
    } catch (error) {
      setMessage('Error adding new planet.');
    }
  };

  return (
    <div>
      <input type="text" value={planet} onChange={(e) => setPlanet(e.target.value)} />
      <button onClick={addPlanet}>Add Planet</button>
      {message && <p>{message}</p>}
    </div>
  );
}

export default App;

Now it's time to put everything together, Space Explorer! Write the complete server-side code to connect to MongoDB, define a schema for planets, save a new planet to the database, and set up the server.

In [ ]:
// index.js (Express server)
const express = require('express');
const cors = require('cors');
const mongoose = require('mongoose');
const initialDBSetup = require('./initialDBSetup');

// Create a new express app
const app = express();

app.use(express.json());
app.use(cors({origin: 'http://localhost:3000'}));

// Connect to MongoDB
mongoose
  .connect('mongodb://127.0.0.1/myDB', { useNewUrlParser: true, useUnifiedTopology: true })
  .then(async () => {
    console.log('connected');
    // initialize DB
    await initialDBSetup();
  });

// Define the schema for planets
// TODO: Define a mongoose schema for planets with properties: name(string), size(string), mass(number), distanceFromSun(string)
const planetSchema = new mongoose.Schema({
  name: String,
  size: String,
  mass: Number,
  distanceFromSun: String
}, { versionKey: false });

// TODO: Compile the planet schema into a model
const Planet = mongoose.model('Planet', planetSchema);

// Set up the POST route for adding a new planet
// TODO: Create an asynchronous POST route handler for '/api/add-planet' that will add a new planet to the database and return the newly created object. Take the name, size, mass, and distance from the request body.

app.post('/api/add-planet', async (req, res) => {
  if (!req.body.name) {
    return res.status(400).send('Name is required');
  }
  // TODO: Create a new planet with the given name req.body.name using Mongoose Model
  const newPlanet = new Planet({
    name: req.body.name,
    size: req.body.size,
    mass: req.body.mass,
    distanceFromSun: req.body.distanceFromSun,
  })
  
  try {
    await newPlanet.save(); 
    // const result = await Planet.find({ name: 'Pluto'}); // Save the new planet and return 200 status code with the newly created object
    res.status(200).json({ message: 'Planet added: ' + result.toString(), data: result });
  } catch (error) {
    res.status(500).json({ message: 'Error adding planet' }); // Return 500 status code with relevant error message in case of failure
  }
  
});

// Start the Express server
// TODO: Define the server port (use environment variable with default), and have the app listen on that port
const port = 5000;
app.listen(port, () => {
  console.log(`Server running at http://localhost:${port}`);
});

In [ ]:
//App.js (React frontend)
import { useEffect, useState } from 'react';
import axios from 'axios';

function App() {
  const [newPlanet, setNewPlanet] = useState({});

  useEffect(() => {
    const addPlanet = async () => {
      const response = await axios.post('/api/add-planet', {
        name: 'Pluto',
        size: 'Small',
        mass: 0.00218,
        distanceFromSun: '5.9 billion km'
      });
      setNewPlanet(response.data);
    };
    addPlanet();
  }, []);

  return (
    <div>
      <h1>Newly Discovered Planet</h1>
      <div>Name: {newPlanet.name}</div>
      <div>Size: {newPlanet.size}</div>
      <div>Mass: {newPlanet.mass}</div>
      <div>Distance: {newPlanet.distanceFromSun}</div>
    </div>
  );
}

export default App;